In [5]:
import time

from selenium import webdriver

from selenium.webdriver.common.by import By

driver = webdriver.Chrome() 

driver.get('http://www.google.com/'); 

time.sleep(5) 

search_box = driver.find_element(By.NAME,'q')

search_box.send_keys('ChromeDriver')

search_box.submit()

time.sleep(5) 



In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/bus-tickets/bangalore-to-madurai?fromCityName=Bangalore&fromCityId=122&srcCountry=India&fromCityType=CITY&toCityName=Madurai&toCityId=126&destCountry=India&toCityType=CITY&onward=24-Jun-2025&doj=24-Jun-2025")

time.sleep(5)


s = driver.find_element(By.CSS_SELECTOR, "ul.srpList__ind-search-styles-module-scss-EOdde")
print(s.text)

driver.quit()


21:45
06:45
9h
27 Seats
(7 Single)
₹999
Onwards
IntrCity SmartBus
A/C Seater / Sleeper (2+1)
4.6
716
New Bus
Toilet
View seats
22:30
06:30
8h
14 Seats
(6 Single)
₹1,402
Onwards
Jabbar Travels
VE A/C Sleeper (2+1)
4.5
975
On Time
View seats
21:15
06:00
8h 45m
20 Seats
(8 Single)
₹1,469
Onwards
IntrCity SmartBus
AC Sleeper (2+1)
4.5
632
New Bus
On Time
View seats
Exclusive
12.5% OFF
21:05
05:35
8h 30m
12 Seats
(6 Single)
₹1,358
₹1,188
Onwards
zingbus plus
A/C Sleeper (2+1)
4.8
115
New Bus
Min. 12.5% OFF on 3 or more seats
View seats
23:15
07:45
8h 30m
13 Seats
(3 Single)
₹1,430
Onwards
Sri Kumaran Travels
Bharat Benz A/C Sleeper (2+1)
4.4
364
New Bus
Toilet
Minimum 10% off on return ticket
View seats
Min. 10% OFF
Best deals of the day
Exclusive
12.5% OFF
22:05
06:35
8h 30m
27 Seats
(11 Single)
₹1,490
₹1,304
Onwards
zingbus plus
A/C Sleeper (2+1)
4.3
527
Try new
10% OFF
23:00
06:45
7h 45m
12 Seats
(6 Single)
₹1,402
₹1,262
Onwards
Dream Line Travels Pvt Ltd
Bharat Benz A/C Sleeper (2+1)
4.

In [8]:
import pandas as pd
import re


data = """
21:45 06:45 9h 29 Seats (8 Single) ₹1,289 Onwards IntrCity SmartBus A/C Seater / Sleeper (2+1) 4.6 716 New Bus Toilet View seats
22:30 06:30 8h 15 Seats (3 Single) ₹1,633 Onwards Jabbar Travels VE A/C Sleeper (2+1) 4.5 975 On Time View seats
21:15 06:00 8h 45m 20 Seats (6 Single) ₹1,539 Onwards IntrCity SmartBus AC Sleeper (2+1) 4.5 632 New Bus On Time View seats
21:10 07:05 9h 55m 26 Seats (7 Single) ₹1,539 Onwards IntrCity SmartBus Bharat Benz A/C Sleeper (2+1) 4.4 665 View seats Try new 5% OFF
"""


lines = data.strip().split("\n")


departure_time = []
arrival_time = []
duration = []
seats = []
price = []
operator = []

for line in lines:
   
    times = re.findall(r'\b\d{2}:\d{2}\b', line)
    departure_time.append(times[0] if len(times) > 0 else None)
    arrival_time.append(times[1] if len(times) > 1 else None)
    
  
    dur = re.search(r'\b\d+h(?: \d{1,2}m)?\b', line)
    duration.append(dur.group() if dur else None)
    

    seat_match = re.search(r'(\d+) Seats', line)
    seats.append(seat_match.group(1) if seat_match else None)

    price_match = re.search(r'₹([\d,]+)', line)
    price.append(price_match.group(1) if price_match else None)
    

    op_match = re.search(r'₹[\d,]+ Onwards ([A-Za-z ]+?)(?: A/C| VE| AC| NON|$)', line)
    operator.append(op_match.group(1).strip() if op_match else None)

df = pd.DataFrame({
    "Departure": departure_time,
    "Arrival": arrival_time,
    "Duration": duration,
    "Seats Available": seats,
    "Price (INR)": price,
    "Operator": operator
})


print(df)


  Departure Arrival Duration Seats Available Price (INR)  \
0     21:45   06:45       9h              29       1,289   
1     22:30   06:30       8h              15       1,633   
2     21:15   06:00   8h 45m              20       1,539   
3     21:10   07:05   9h 55m              26       1,539   

                        Operator  
0              IntrCity SmartBus  
1                 Jabbar Travels  
2              IntrCity SmartBus  
3  IntrCity SmartBus Bharat Benz  


In [10]:
import sqlite3
import pandas as pd
import re


data_text = """
21:45 06:45 9h 29 Seats (8 Single) ₹1,289 Onwards IntrCity SmartBus A/C Seater / Sleeper (2+1) 4.6 716 New Bus Toilet View seats
22:30 06:30 8h 15 Seats (3 Single) ₹1,633 Onwards Jabbar Travels VE A/C Sleeper (2+1) 4.5 975 On Time View seats
21:15 06:00 8h 45m 20 Seats (6 Single) ₹1,539 Onwards IntrCity SmartBus AC Sleeper (2+1) 4.5 632 New Bus On Time View seats
21:10 07:05 9h 55m 26 Seats (7 Single) ₹1,539 Onwards IntrCity SmartBus Bharat Benz A/C Sleeper (2+1) 4.4 665 View seats Try new 5% OFF
"""

lines = data_text.strip().split("\n")

bus_data = []
bus_id = 1

for line in lines:
    times = re.findall(r'\b\d{2}:\d{2}\b', line)
    departure = times[0] if len(times) > 0 else None
    arrival = times[1] if len(times) > 1 else None
    
    dur = re.search(r'\b\d+h(?: \d{1,2}m)?\b', line)
    duration = dur.group() if dur else None
    
    seat_match = re.search(r'(\d+) Seats', line)
    seats_total = int(seat_match.group(1)) if seat_match else None
    
    price_match = re.search(r'₹([\d,]+)', line)
    price = int(price_match.group(1).replace(',', '')) if price_match else None
    
    op_match = re.search(r'₹[\d,]+ Onwards ([A-Za-z ]+?)(?: A/C| VE| AC| NON|$)', line)
    operator = op_match.group(1).strip() if op_match else None
    
    bus_data.append((bus_id, departure, arrival, duration, seats_total, price, operator))
    bus_id += 1

conn = sqlite3.connect("busdata.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS buses (
    Bus_ID INTEGER PRIMARY KEY,
    Departure_Time TEXT,
    Arrival_Time TEXT,
    Duration TEXT,
    Seats_Total INTEGER,
    Price INTEGER,
    Operator TEXT
)
""")

cursor.executemany("INSERT OR IGNORE INTO buses VALUES (?, ?, ?, ?, ?, ?, ?)", bus_data)
conn.commit()


df = pd.read_sql_query("SELECT * FROM buses WHERE Price < 1600", conn)
print(df)

conn.close()


   Bus_ID Departure_Time Arrival_Time Duration  Seats_Total  Price  \
0       1          21:45        06:45       9h           29   1289   
1       3          21:15        06:00   8h 45m           20   1539   
2       4          21:10        07:05   9h 55m           26   1539   

                        Operator  
0              IntrCity SmartBus  
1              IntrCity SmartBus  
2  IntrCity SmartBus Bharat Benz  
